# Calculate Metrics

**Description:** This notebook compares different groups of experiments, using the metrics described in each cell.

**Real Time**
* Burst duration
* Rise duration
* Decay duration
* Interval duration
* SNA duration

**Developmental Time**
* At Onset
* At Hatching
* Per Burst
* CDF

**Other Features**
* Amplitude
* AUC Bins
* Local Peaks

## Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd

from snazzy_analysis import Experiment, Group, FrequencyAnalysis, utils, myplots, TracePhases

## Load Data

In [ ]:
wt_folder = "25C"
wt_config = ["20240611_25C", "20240919_25C", "20250404_25C"]

rdl_folder = "rdl"
rdl_config = ["20250211_rdl", "20250212_rdl"]

wt_experiments = {}
for exp in wt_config:
    exp_path = Path.cwd().parent.joinpath("data", wt_folder, exp)
    wt_experiments[exp] = Experiment(exp_path)

rdl_experiments = {}
for exp in rdl_config:
    exp_path = Path.cwd().parent.joinpath("data", rdl_folder, exp)
    rdl_experiments[exp] = Experiment(exp_path)

wt = Group("WT", wt_experiments)
rdl = Group("Rdl-", rdl_experiments)

groups = [wt, rdl]

## Configure Plot Style

Customize the Matplotlib plots with rcParams. For a full list of parameters see [matplotlib.rcParams](https://matplotlib.org/stable/api/matplotlib_configuration_api.html#default-values-and-styling)

In [ ]:
myplots.set_darkbg(False)

num_episodes = 15

episode_metrics_rc = {
    "figure.figsize": (6, 3),
    "figure.frameon": False,
    "lines.linewidth": 1,
    "font.family": "Arial",
    "font.size": 20,
    "xtick.major.size": 10,  # length
    "xtick.major.width": 2,
    "ytick.major.size": 10,  # length
    "ytick.major.width": 2,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.labelpad": 10,
    "axes.linewidth": 2.5,
}

num_groups = len(groups)
group_metrics_rc = {
    "figure.figsize": (6, 3),
    "figure.frameon": False,
    "lines.linewidth": 2.25,
    "font.family": "Arial",
    "font.size": 20,
    "xtick.major.size": 10,  # length
    "xtick.major.width": 2,
    "ytick.major.size": 10,  # length
    "ytick.major.width": 2,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.labelpad": 10,
    "axes.linewidth": 2.5,
}

# Overview

## Sample Size

Prints a table displaying the total sample size for each group, plus the sample size for each experiment in a group. 

In [ ]:
data = {"Group": [], "Exp": [], "Emb": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            data["Group"].append(group.name)
            data["Exp"].append(exp.name)
            data["Emb"].append(emb)

data = pd.DataFrame(data)

# calculate total Ns per group
num_per_exp = data.groupby(["Group", "Exp"]).size().reset_index(name="N")
num_per_group = data.groupby("Group").size().reset_index(name="N")
num_per_group["Exp"] = "**Total**"
num_per_group = num_per_group[["Group", "Exp", "N"]]
combined = pd.concat([num_per_exp, num_per_group], ignore_index=True)
flipped = combined[::-1]

print(flipped.to_string(index=False))

# Real Time

## Burst Duration

In [ ]:
data = {"Group": [], "Burst Duration (min)": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = emb.trace
            for i, duration in zip(range(num_episodes), trace.peak_durations):
                data["Group"].append(group.name)
                data["Burst Duration (min)"].append(duration / 60)
                data["Burst #"].append(i + 1)

data = pd.DataFrame(data)

myplots.plot_pointplot(
    data,
    x="Burst #",
    y="Burst Duration (min)",
    category="Group",
    rc=episode_metrics_rc,
    linestyle="-.",
)

## Rise Duration

In [ ]:
data = {"Group": [], "Rise Duration (sec)": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = emb.trace
            for i, rise in zip(range(num_episodes), trace.peak_rise_times):
                data["Group"].append(group.name)
                data["Rise Duration (sec)"].append(rise)
                data["Burst #"].append(i + 1)

data = pd.DataFrame(data)

myplots.plot_pointplot(
    data,
    x="Burst #",
    y="Rise Duration (sec)",
    category="Group",
    rc=episode_metrics_rc,
    linestyle="-.",
)

## Decay Duration

In [ ]:
data = {"Group": [], "Decay Duration (sec)": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = emb.trace
            for i, decay in zip(range(num_episodes), trace.peak_decay_times):
                data["Group"].append(group.name)
                data["Decay Duration (sec)"].append(decay)
                data["Burst #"].append(i + 1)

data = pd.DataFrame(data)

myplots.plot_pointplot(
    data,
    x="Burst #",
    y="Decay Duration (sec)",
    category="Group",
    rc=episode_metrics_rc,
    linestyle="-.",
)

## Interval Duration

In [ ]:
data = {"Group": [], "Interval Duration (min)": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = emb.trace
            for i, interval in zip(range(num_episodes), trace.peak_intervals):
                data["Group"].append(group.name)
                data["Interval Duration (min)"].append(interval / 60)
                data["Burst #"].append(i + 1)

data = pd.DataFrame(data)

myplots.plot_pointplot(
    data,
    x="Burst #",
    y="Interval Duration (min)",
    category="Group",
    rc=episode_metrics_rc,
    linestyle="-.",
)

## SNA Duration

In [ ]:
data = {"Group": [], "Duration (min)": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = emb.trace
            if len(trace.peak_idxes) == 0:
                continue
            data["Group"].append(group.name)
            duration = (trace.time[trace.trim_idx] - trace.peak_times[0]) / 60
            data["Duration (min)"].append(duration)

data = pd.DataFrame(data)

myplots.plot_pointplot(
    data, x="Group", y="Duration (min)", category="Group", rc=episode_metrics_rc
)

# Developmental Time

## Dev Time at Onset

In [ ]:
data = {"Group": [], "Dev Time": []}

for group in groups:
    for exp in group.experiments.values():
        num = 0
        for emb in exp.embryos:
            num = num + 1
            trace = emb.trace
            if len(trace.peak_idxes) == 0:
                continue
            time_first_peak = trace.peak_times[0]
            dev_time_first_peak = emb.get_DT_from_time(time_first_peak)
            data["Dev Time"].append(dev_time_first_peak)
            data["Group"].append(group.name)

data = pd.DataFrame(data)

myplots.plot_pointplot(
    data, x="Group", y="Dev Time", category="Group", rc=group_metrics_rc
)

## Dev Time at Hatching

In [ ]:
data = {"Group": [], "Dev Time": []}

for group in groups:
    for exp in group.experiments.values():
        num = 0
        for emb in exp.embryos:
            num = num + 1
            trace = emb.trace
            if len(trace.peak_idxes) == 0:
                continue
            time_hatching = trace.time[trace.trim_idx]
            dev_time_first_peak = emb.get_DT_from_time(time_hatching)

            data["Dev Time"].append(dev_time_first_peak)
            data["Group"].append(group.name)

data = pd.DataFrame(data)

myplots.plot_pointplot(
    data,
    x="Group",
    y="Dev Time",
    category="Group",
    rc=group_metrics_rc,
)

## Dev Time Per Burst

In [ ]:
data = {"Group": [], "Dev Time": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb_n, emb in enumerate(exp.embryos):
            trace = emb.trace
            if len(trace.peak_idxes) == 0:
                continue
            for i, t in zip(range(num_episodes), trace.peak_times):
                data["Group"].append(group.name)
                data["Dev Time"].append(emb.get_DT_from_time(t))
                data["Burst #"].append(i + 1)

data = pd.DataFrame(data)

myplots.plot_pointplot(
    data, x="Burst #", y="Dev Time", category="Group", rc=episode_metrics_rc
)

## CDF Dev Time Per Burst

In [ ]:
data = {"Group": [], "Dev Time": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = emb.trace
            if len(trace.peak_idxes) == 0:
                continue
            dev_times = [emb.get_DT_from_time(t) for t in trace.peak_times]
            data["Dev Time"].extend(dev_times)
            data["Group"].extend([group.name] * len(dev_times))

data = pd.DataFrame(data)

myplots.plot_cdf(data, "Dev Time", category="Group", rc=group_metrics_rc)

# Other Features

## Burst Amplitude

In [ ]:
data = {"Group": [], "Amplitude": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            t = emb.trace
            for i, amp in zip(range(num_episodes), t.peak_amplitudes):
                data["Amplitude"].append(amp)
                data["Group"].append(group.name)
                data["Burst #"].append(i)

data = pd.DataFrame(data)

myplots.plot_pointplot(
    data, x="Burst #", y="Amplitude", category="Group", rc=episode_metrics_rc
)

## AUC Bins

In [ ]:
data = {"Group": [], "AUC": [], "Bin": []}

n_bins = 5
first_bin = 2
bin_width = 0.2

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = emb.trace
            dev_time_at_peaks = emb.get_DT_from_time(trace.peak_times)
            bins = [first_bin + j * bin_width for j in range(n_bins)]
            bin_idxs = utils.split_in_bins(dev_time_at_peaks, bins)
            data["Group"].extend([str(group.name)] * len(trace.peak_aucs))
            data["AUC"].extend(trace.peak_aucs)
            data["Bin"].extend(bin_idxs)

data = pd.DataFrame(data)

# make x labels
bins.append(first_bin + bin_width * n_bins)
x_labels = [f"{s}~{e}" for (s, e) in zip(bins[:-1], bins[1:])]

myplots.plot_pointplot(
    data, x="Bin", y="AUC", category="Group", rc=episode_metrics_rc, xlabels=x_labels
)

## Local Peaks Per Burst

In [ ]:
data = {"Group": [], "# Local Peaks": [], "Burst #": []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = emb.trace
            local_peaks = trace.compute_local_peaks(height=0.03, prominence=0.02)
            for i, lp in zip(range(15), local_peaks):
                data["# Local Peaks"].append(lp)
                data["Burst #"].append(i + 1)
                data["Group"].append(group.name)

data = pd.DataFrame(data)

myplots.plot_pointplot(
    data, x="Burst #", y="# Local Peaks", category="Group", rc=episode_metrics_rc
)

## Burst Rates, Grouped by Bins

In [ ]:
data = {"Group": [], "Rate": [], "Bin": []}

n_bins = 40
first_bin = 1
bin_width = 0.2
bins = np.arange(start=first_bin, stop=first_bin + n_bins * bin_width, step=bin_width)

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = emb.trace
            time_bins, idx_offset = emb.get_time_bins(bins)
            time_intervals = np.diff(time_bins) / 3600
            time_bin_idxs = utils.split_in_bins(trace.peak_times, time_bins) - 1
            freqs = Counter(time_bin_idxs)
            rates = {k + idx_offset: v / time_intervals[k] for k, v in freqs.items()}
            data["Group"].extend([str(group.name)] * len(rates))
            data["Rate"].extend(rates.values())
            data["Bin"].extend(rates.keys())

data = pd.DataFrame(data)

# make x labels
first_used_bin = data["Bin"].min()
last_used_bin = data["Bin"].max()
values_to_add = list(range(first_used_bin, last_used_bin))
missing_values = [val for val in values_to_add if val not in data["Bin"].values]
if missing_values:
    data = pd.concat([data, pd.DataFrame({"Bin": missing_values})])
x_labels = [
    round(bins[first_used_bin] + bin_width * i, 1)
    for i in range(last_used_bin - first_used_bin + 1)
]

myplots.plot_pointplot(
    data, x="Bin", y="Rate", category="Group", rc=episode_metrics_rc, xlabels=x_labels
)

## Burst Intervals, Grouped by Bins

In [ ]:
data = {"Group": [], "Interval": [], "Bin": []}

n_bins = 40
first_bin = 1
bin_width = 0.2
bins = [first_bin + j * bin_width for j in range(n_bins)]

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = emb.trace
            if len(trace.peak_idxes) == 0:
                continue
            lin_dev_time = emb.lin_developmental_time
            # skip the last peak because when we calculate peak intervals we loose a peak
            dev_time_at_peaks = lin_dev_time[trace.peak_idxes[:-1]]
            bin_idxs = utils.split_in_bins(dev_time_at_peaks, bins)
            data["Group"].extend([str(group.name)] * len(trace.peak_intervals))
            data["Interval"].extend(trace.peak_intervals / 60)
            data["Bin"].extend(bin_idxs)

data = pd.DataFrame(data)

# make x labels
first_used_bin = data["Bin"].min()
last_used_bin = data["Bin"].max()
values_to_add = list(range(first_used_bin, last_used_bin))
missing_values = [val for val in values_to_add if val not in data["Bin"].values]
if missing_values:
    data = pd.concat([data, pd.DataFrame({"Bin": missing_values})])
x_labels = [
    round(bins[first_used_bin] + bin_width * i, 1)
    for i in range(last_used_bin - first_used_bin + 1)
]

myplots.plot_pointplot(
    data,
    x="Bin",
    y="Interval",
    category="Group",
    rc=episode_metrics_rc,
    xlabels=x_labels,
)

## DFF signal overlayed with low passed signal

In [ ]:
emb = wt.experiments["20240919_25C"].embryos[9]
print(emb.name)
trace = emb.trace

time, dff = trace.preprocess_dff()

low_pass_dff = FrequencyAnalysis.apply_lopass_filter(dff, 0.006)

overlay_rc = group_metrics_rc.copy()
overlay_rc["figure.figsize"] = (15, 3)
myplots.plot_trace_with_overlay(
    time,
    dff,
    low_pass_dff,
    overlay_rc,
    color="#0273b2ff",
    xmax=250,
    ymax=1,
    yinterval=0.5,
)

## Low frequency and high frequency events

In [ ]:
import matplotlib.pyplot as plt
import scipy.signal as spsig

emb = wt.experiments["20240919_25C"].embryos[9]
trace = emb.trace

local_peaks, _ = spsig.find_peaks(
    trace.dff[: trace.trim_idx], height=0.09, prominence=0.03
)
local_peaks = [
    lp for lp in local_peaks if not np.any(np.abs(trace.peak_idxes - lp) <= 10)
]
amp_local_peaks = trace.dff[local_peaks]
amp_bursts = trace.dff[trace.peak_idxes]

fig, ax = plt.subplots(figsize=(19, 4))
ax.plot(trace.dff[: trace.trim_idx])
ax.plot(local_peaks, amp_local_peaks, "r.")
ax.plot(trace.peak_idxes, amp_bursts, "k.")
ax.set_title(emb.name)
plt.show()

## Number of high frequency events in first hour and last hour of development

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_rel

trace = wt.experiments["20240611_25C"].embryos[4].trace


def get_hf_magnitude(signal, freq_cutoff, fs=1 / 6):
    N = len(signal)
    freqs = np.fft.rfftfreq(N, 1 / fs)
    fft = np.fft.rfft(signal)
    mask = freqs > freq_cutoff
    filtered_fft = fft * mask
    return np.sum(np.abs(filtered_fft))


data = {"condition": [], "value": [], "emb_id": [], "experiment": []}

for experiment in wt.experiments.values():
    for emb in experiment.embryos:
        trace = emb.trace
        first_peak = trace.peak_idxes[0]
        last_peak = trace.peak_idxes[-2]
        early_magnitude = get_hf_magnitude(
            trace.dff[first_peak - 50 : first_peak + 550], 0.01
        )
        late_magnitude = get_hf_magnitude(trace.dff[last_peak - 600 : last_peak], 0.01)
        data["experiment"].append(experiment.name)
        data["experiment"].append(experiment.name)
        data["emb_id"].append(experiment.name + emb.name)
        data["emb_id"].append(experiment.name + emb.name)
        data["value"].append(early_magnitude)
        data["condition"].append("hf_early")
        data["value"].append(late_magnitude)
        data["condition"].append("hf_late")

df = pd.DataFrame(data)

hf_late = df[df["condition"] == "hf_late"]["value"].values
hf_early = df[df["condition"] == "hf_early"]["value"].values
stat, pval = ttest_rel(hf_late, hf_early)
print(f"Paired t-test: t={stat:.2f}, p={pval}")

fig, ax = plt.subplots(figsize=(2, 3))

sns.barplot(
    data=data,
    x="condition",
    y="value",
    ax=ax,
)

for emb_id, pair in df.groupby("emb_id"):
    pair_sorted = pair.sort_values("condition")
    plt.plot(
        pair_sorted["condition"], pair_sorted["value"], marker=".", color="k", alpha=0.7
    )

ax.set_title("Early and Late High Frequency Events Magnitude")
ax.set_ylabel("Magnitude")
plt.show()

## Binned count of high frequency events

In [ ]:
data = {
    "emb_id": [],
    "bin_idx": [],
    "value": [],
    "type": [],
}


def find_hf_events(trace):
    local_peaks, _ = spsig.find_peaks(
        trace.aligned_dff[: trace.trim_idx], height=0.1, prominence=0.05
    )
    return np.array(
        [lp for lp in local_peaks if not np.any(np.abs(trace.peak_idxes - lp) <= 10)]
    )


def hits_per_bin(values, bin_width, num_bins):
    bin_idx = values // bin_width
    return np.bincount(bin_idx, minlength=num_bins)


n_bins = 15
first_bin = 0
bin_width = 250
bins = np.arange(start=first_bin, stop=first_bin + n_bins * bin_width, step=bin_width)

for exp in wt.experiments.values():
    onset = 0
    for emb in exp.embryos:
        trace = emb.trace

        hf_peaks = find_hf_events(trace)

        hf_bin_count = hits_per_bin(hf_peaks, bin_width, n_bins)
        onset = trace.peak_bounds_indices[0][0]
        onset = onset - 300 if onset > 300 else onset
        lf_bin_count = hits_per_bin(trace.peak_idxes - onset, bin_width, n_bins)

        # hf entries
        data["emb_id"].extend([exp.name + emb.name] * len(bins))
        data["bin_idx"].extend(bins)
        data["value"].extend(hf_bin_count)
        data["type"].extend(["hf_event"] * len(bins))
        # lf entries
        data["emb_id"].extend([exp.name + emb.name] * len(bins))
        data["bin_idx"].extend(bins)
        data["value"].extend(lf_bin_count)
        data["type"].extend(["lf_event"] * len(bins))

data = pd.DataFrame(data)

x_labels = [b // 10 if i % 3 == 0 else "" for i, b in enumerate(bins)]

myplots.plot_pointplot(
    data,
    x="bin_idx",
    y="value",
    category="type",
    rc=episode_metrics_rc,
    xlabels=x_labels,
)

plt.tight_layout()

## Phase 1 End

In [ ]:
emb = wt.experiments["20240919_25C"].embryos[5]
trace = emb.trace

tp = TracePhases(trace)

features = tp.phase1_features(hf_cutoff=0.01)
dm = tp.dist_matrix(features)
thres = tp.feature_thres(dm, num_classes=2)
change_index = tp.segment_distance_matrix_forward(dm, thres)
tp.plot_phase_change(dm, change_index, features)

## Developmental time at first burst, phase 1 end, and hatching

In [ ]:
data = {"emb_id": [], "metric": [], "value": []}

for exp in wt.experiments.values():
    for emb in exp.embryos:

        data["emb_id"].append(exp.name + emb.name)
        data["metric"].append("dt_first_peak")
        initial_dt = emb.get_DT_from_time(emb.trace.time[emb.trace.peak_idxes[0]])
        data["value"].append(initial_dt)

        data["emb_id"].append(exp.name + emb.name)
        data["metric"].append("dt_phase1_end")
        tp = TracePhases(emb.trace)
        p1_end = tp.get_phase1_end(freq=0.010)
        p1_end_time = emb.trace.time[p1_end]
        p1_end_dt = emb.get_DT_from_time(p1_end_time)
        data["value"].append(p1_end_dt)

        data["emb_id"].append(exp.name + emb.name)
        data["metric"].append("dt_hatching")
        final_dt = emb.get_DT_from_time(emb.trace.time[emb.trace.trim_idx])
        data["value"].append(final_dt)

x_order = ["dt_first_peak", "dt_phase1_end", "dt_hatching"]
palette = sns.color_palette("pastel", n_colors=len(x_order))
color_map = dict(zip(x_order, palette))

fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(
    data=data,
    x="metric",
    y="value",
    hue="metric",
    order=x_order,
    ax=ax,
)

for i, emb_id in enumerate(data["emb_id"]):
    x = data["metric"][3 * i : 3 * i + 3]
    y = data["value"][3 * i : 3 * i + 3]
    ax.plot(x, y, color="black", linewidth=0.75)

dts_first_peak = data["value"][::3]
ax.plot(
    ["dt_first_peak"] * len(dts_first_peak),
    dts_first_peak,
    marker="o",
    markerfacecolor=color_map["dt_first_peak"],
    markeredgecolor="k",
)

dts_phase1_end = data["value"][1::3]
ax.plot(
    ["dt_phase1_end"] * len(dts_first_peak),
    dts_phase1_end,
    marker="o",
    markerfacecolor=color_map["dt_phase1_end"],
    markeredgecolor="k",
)

dts_hatching = data["value"][2::3]
ax.plot(
    ["dt_hatching"] * len(dts_first_peak),
    dts_hatching,
    marker="o",
    markerfacecolor=color_map["dt_hatching"],
    markeredgecolor="k",
)

plt.ylim(1.5, 3)
plt.title("WT embryos")
plt.tight_layout()
plt.show()